In [ ]:
import cv2
import numpy as np

#getting the webcam feed and saving the image and video in variables
cap = cv2.VideoCapture(1)
imgTarget= cv2.imread('markerimg.jpg')
video = cv2.VideoCapture('displayvideo1.mp4')

detection =False
frameCounter =0

#gets the first frame of the video
success,dispVideo = video.read()

#resizing the video frame to match the image target
hT,wT,cT = imgTarget.shape
dispVideoResized= cv2.resize(dispVideo,(wT,hT))

#implementing oriented FAST algorithm
orb = cv2.ORB_create(nfeatures = 1000)
# creating keypoints and descriptors
kp1,des1 = orb.detectAndCompute(imgTarget,None)
imgTargetWithKeyPoints = cv2.drawKeypoints(imgTarget,kp1,None)

while(True):
    #reading the webcam feed for everyframe
    sucess,WebcamFeed = cap.read()
    AugmentedImage = WebcamFeed.copy()
    
    kp2,des2 = orb.detectAndCompute(WebcamFeed,None)
    WebcamFeed = cv2.drawKeypoints(WebcamFeed,kp2,None)
    
    # if there is no image target detected reset the video i.e start the video from the begining and set the frame counter to 0
    if detection == False:
        video.set(cv2.CAP_PROP_POS_FRAMES,0)
        frameCounter = 0
    else:
        # if the video has completed loop back again
        if frameCounter == video.get(cv2.CAP_PROP_FRAME_COUNT):
            video.set(cv2.CAP_PROP_POS_FRAMES,0)
            frameCounter = 0
        success,dispVideo = video.read()
        dispVideoResized= cv2.resize(dispVideo,(wT,hT))
    #matching the target image with the webcam feed by comparing the descriptor values using K nearest neighbours
    bruteForce = cv2.BFMatcher()
    # setting 2 output values  becaues two images are compared
    matches = bruteForce.knnMatch(des1,des2,k=2)
    good =[]
    # since K=2, the two vlaues in matches can be unpacked
    for m,n in matches:
        #comparing the distance between the descriptor values of the respective images and classifying them as good 
        if m.distance<0.75*n.distance:
            good.append(m)
    print(len(good))
    imgFeatures = cv2.drawMatches(imgTarget,kp1,WebcamFeed,kp2,good,None,flags= 2)
    
    if len(good)>15:
        detection = True
        #creating source points and destination points matrix to measure homography (to perform necessary transformations)
        srcpt= np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        despt= np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        
        #using ransac algorithm to eliminate any outliers
        matrix,mask = cv2.findHomography(srcpt,despt,cv2.RANSAC,5)
        print(matrix)
        
        #drawing the bounding box around the target image using perspective transform
        pts = np.float32([[0,0],[0,hT],[wT,hT],[wT,0]]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,matrix)
        #configuring the shape and colour of the bounding box
        img2 = cv2.polylines(WebcamFeed,[np.int32(dst)],True,(255,0,255),3)
        #warping the webcam feed to the transformed imagetarget in the scene
        imgWarp = cv2.warpPerspective(dispVideoResized,matrix,(WebcamFeed.shape[1],WebcamFeed.shape[0]))
        
        #creating a mask to overlay the video on the target image        
        maskNew = np.zeros((WebcamFeed.shape[0],WebcamFeed.shape[1]),np.uint8)
        cv2.fillPoly(maskNew,[np.int32(dst)],(255,255,255))
        mskInv = cv2.bitwise_not(maskNew)
        AugmentedImage = cv2.bitwise_and(AugmentedImage,AugmentedImage,mask=mskInv)
        AugmentedImage = cv2.bitwise_or(imgWarp,AugmentedImage)
        

           
        
        
    #cv2.imshow('border',img2)
    #cv2.imshow('imgWarp',imgWarp)
    #cv2.imshow('maskNew',maskNew)
    cv2.imshow('imgFeatures',imgFeatures)
    cv2.imshow('AugmentedImage',AugmentedImage)  
    #cv2.imshow('imgTarget',imgTarget)
    #cv2.imshow('imgTargetWithKeyPoints',imgTargetWithKeyPoints)
    #cv2.imshow('dispVideo',dispVideoResized)
    #cv2.imshow ('WebcamFeed',WebcamFeed)
    cv2.waitKey(1)
    frameCounter +=1



5
3
1
1
0
2
2
1
3
4
2
1
0
1
1
3
1
2
2
1
1
1
1
1
1
1
5
8
3
2
5
2
1
3
2
4
4
6
3
7
3
4
12
21
[[ 9.47867039e-01  3.45150692e-01  1.94578103e+02]
 [-1.22504899e-01  1.11434055e+00  1.36942554e+02]
 [-2.13300495e-04  6.75655555e-04  1.00000000e+00]]
27
[[ 8.71696633e-01  5.78373929e-02  2.01997265e+02]
 [-1.31851232e-01  9.48258160e-01  1.32369142e+02]
 [-2.71157658e-04 -5.00426624e-05  1.00000000e+00]]
23
[[ 8.77799003e-01  5.06163181e-02  1.98360964e+02]
 [-1.36082927e-01  9.44675075e-01  1.27011629e+02]
 [-2.91389741e-04 -6.46395950e-05  1.00000000e+00]]
30
[[ 9.55013742e-01  1.30699456e-01  1.92044734e+02]
 [-1.18064012e-01  1.05947072e+00  1.17172506e+02]
 [-2.12823493e-04  2.05500878e-04  1.00000000e+00]]
30
[[ 9.29131191e-01  4.15273760e-02  1.93335267e+02]
 [-1.11171152e-01  1.01953325e+00  1.10958796e+02]
 [-2.57615709e-04 -1.94971181e-05  1.00000000e+00]]
32
[[ 9.30648317e-01  5.68215242e-02  1.92707100e+02]
 [-1.06562238e-01  1.06853040e+00  1.05098689e+02]
 [-2.93663191e-04  9.20

 [-1.50720756e-04  1.39095410e-05  1.00000000e+00]]
29
[[ 9.78468900e-01 -4.76252268e-02  1.57216688e+02]
 [-9.66497267e-02  1.00927999e+00  7.47648225e+01]
 [-1.97900565e-04 -2.43457460e-04  1.00000000e+00]]
27
[[ 1.03668821e+00  8.41660880e-02  1.48956680e+02]
 [-9.16130836e-02  1.10572054e+00  6.92411185e+01]
 [-1.39893021e-04  8.38087900e-05  1.00000000e+00]]
28
[[ 1.09709028e+00  1.34548954e-01  1.45144734e+02]
 [-7.21012365e-02  1.15433300e+00  6.44248189e+01]
 [-3.23511307e-05  2.09043152e-04  1.00000000e+00]]
26
[[ 9.90329824e-01  4.55539493e-02  1.54552780e+02]
 [-9.82012621e-02  1.07104744e+00  6.70521055e+01]
 [-2.09420307e-04  1.06335373e-05  1.00000000e+00]]
30
[[ 1.00929117e+00  3.42124972e-02  1.54451831e+02]
 [-9.28185545e-02  1.06255595e+00  6.57184285e+01]
 [-1.70893287e-04 -7.10348575e-05  1.00000000e+00]]
31
[[ 1.03777275e+00  7.14717701e-02  1.54140755e+02]
 [-8.76440763e-02  1.09796128e+00  6.45445676e+01]
 [-1.32248586e-04  6.51977483e-05  1.00000000e+00]]
30
[[ 

21
[[ 1.48836795e+00  1.91314851e-01  3.03246446e+01]
 [-1.77513175e-01  1.61322622e+00  4.33556300e+01]
 [-2.75533411e-04  2.03657629e-04  1.00000000e+00]]
20
[[ 1.52551262e+00  1.32221055e-01  3.10154804e+01]
 [-1.56771702e-01  1.65067722e+00  4.01205363e+01]
 [-1.65706515e-04  1.27631545e-04  1.00000000e+00]]
28
[[ 1.56157176e+00  1.60346355e-01  2.37722909e+01]
 [-1.52124217e-01  1.66315869e+00  4.38308674e+01]
 [-1.19901470e-04  1.94021153e-04  1.00000000e+00]]
25
[[ 1.55193966e+00  1.66205673e-01  2.00649028e+01]
 [-1.59415998e-01  1.64752514e+00  4.63379450e+01]
 [-1.30689548e-04  1.59418690e-04  1.00000000e+00]]
23
[[ 1.50288871e+00  8.08069638e-02  2.56661301e+01]
 [-1.66695203e-01  1.57320834e+00  5.08701623e+01]
 [-1.58456171e-04 -6.83830188e-05  1.00000000e+00]]
27
[[ 1.53810936e+00  1.63255926e-01  1.79298000e+01]
 [-1.74061596e-01  1.68140831e+00  4.74340260e+01]
 [-1.96485881e-04  2.86232099e-04  1.00000000e+00]]
32
[[ 1.49344050e+00  1.30606710e-01  2.00707863e+01]
 [-1

21
[[ 1.15335694e+00 -1.40946634e-01  2.77599276e+02]
 [ 6.17348429e-02  1.19129757e+00  2.93027678e+01]
 [-8.73570008e-05 -5.05672903e-05  1.00000000e+00]]
31
[[ 1.17941249e+00 -8.72446191e-02  2.71256618e+02]
 [ 3.95905394e-02  1.21505835e+00  3.73916578e+01]
 [-6.57130955e-05  1.62623999e-06  1.00000000e+00]]
19
[[ 1.27936462e+00 -3.39543310e-02  2.64314072e+02]
 [ 5.75523086e-02  1.28176932e+00  3.61763270e+01]
 [ 7.09759600e-05  7.34054579e-05  1.00000000e+00]]
19
[[ 1.20702916e+00 -1.10708982e-01  2.67637875e+02]
 [ 2.09815558e-02  1.21473778e+00  4.57452292e+01]
 [-2.12369070e-05 -1.19848355e-04  1.00000000e+00]]
33
[[ 1.17741423e+00 -7.59359381e-02  2.67727076e+02]
 [ 1.24545442e-02  1.21552090e+00  4.94246772e+01]
 [-7.62997080e-05 -4.50650536e-05  1.00000000e+00]]
30
[[ 1.17567355e+00  6.41511959e-02  2.64303646e+02]
 [-2.03746767e-03  1.29900973e+00  5.08819007e+01]
 [-1.48874516e-04  3.07593945e-04  1.00000000e+00]]
25
[[ 1.17184065e+00 -4.05752177e-02  2.65346159e+02]
 [ 2

36
[[ 1.17418759e+00  1.03914759e-01  1.49550495e+02]
 [-4.63751228e-02  1.31540665e+00  6.68862175e+01]
 [-2.30961989e-04  2.91744407e-04  1.00000000e+00]]
23
[[ 1.23794885e+00  7.76184576e-02  1.46604233e+02]
 [-3.48039965e-02  1.29697405e+00  7.04874330e+01]
 [-1.04136873e-04  1.74313126e-04  1.00000000e+00]]
29
[[ 1.19847184e+00  8.55369519e-02  1.48564518e+02]
 [-5.62048564e-02  1.30873745e+00  7.34470738e+01]
 [-1.88174001e-04  2.22197807e-04  1.00000000e+00]]
32
[[ 1.21621341e+00  6.49652568e-02  1.51400706e+02]
 [-5.69598853e-02  1.32021412e+00  7.58322490e+01]
 [-1.41848149e-04  1.76095273e-04  1.00000000e+00]]
28
[[ 1.18320204e+00  2.97781211e-02  1.53828216e+02]
 [-5.89329017e-02  1.25889720e+00  7.76683404e+01]
 [-1.85477922e-04 -8.12852950e-06  1.00000000e+00]]
32
[[ 1.21450523e+00  1.06084079e-01  1.52213728e+02]
 [-5.59105621e-02  1.33179816e+00  7.55070900e+01]
 [-1.56860832e-04  2.38662000e-04  1.00000000e+00]]
28
[[ 1.22865756e+00  9.70466869e-02  1.53287216e+02]
 [-5

11
12
20
[[ 1.38746202e+00  2.75239775e-01 -2.81998325e+02]
 [-1.02678281e-01  1.82056674e+00  8.36971955e+01]
 [ 1.38518777e-03  1.51708794e-03  1.00000000e+00]]
18
[[ 2.96691819e-01  4.57323155e-02 -4.91269053e+01]
 [-3.88130566e-01  4.86258319e-01  1.24128257e+02]
 [-1.96046642e-03 -2.84299655e-04  1.00000000e+00]]
11
13
6
7
5
5
7
6
4
4
2
6
4
5
6
3
2
3
3
1
0
2
3
0
2
1
1
1
2
0
1
2
1
1
0
2
1
0
2
2
2
5
3
0
2
5
4
13
15
14
10
12
12
9
5
2
5
5
3
9
7
15
11
13
6
10
8
8
9
18
[[1.46055059e+00 2.64371488e-01 1.33455946e+02]
 [4.68822529e-02 1.70856154e+00 1.12125588e+02]
 [2.38030312e-04 9.04511588e-04 1.00000000e+00]]
13
17
[[ 1.31235136e+00  9.90882388e-02  1.35407352e+02]
 [-2.82308462e-03  1.50868871e+00  1.22405851e+02]
 [ 3.34029534e-05  5.46407467e-04  1.00000000e+00]]
14
16
[[ 1.11807237e+00  6.53133378e-02  1.28238469e+02]
 [-1.08217063e-01  1.25269605e+00  1.37819484e+02]
 [-3.48781774e-04  1.47804653e-04  1.00000000e+00]]
13
14
13
11
18
[[ 1.19143632e+00  7.20798338e-02  1.20505587e+

27
[[ 1.18187916e+00  3.62536384e-02  1.35693689e+02]
 [-8.27693135e-02  1.27704961e+00  1.07959621e+02]
 [-2.01271805e-04  1.30068163e-04  1.00000000e+00]]
26
[[ 1.17540325e+00  7.63309715e-02  1.34227111e+02]
 [-8.44933155e-02  1.28932724e+00  1.07300438e+02]
 [-2.07316828e-04  1.77211068e-04  1.00000000e+00]]
28
[[ 1.16418055e+00  3.38360246e-02  1.36027003e+02]
 [-8.40254675e-02  1.25572808e+00  1.07701283e+02]
 [-2.17514882e-04  7.11081690e-05  1.00000000e+00]]
33
[[ 1.17336695e+00  5.99521511e-02  1.33712357e+02]
 [-7.91689206e-02  1.27029918e+00  1.06095215e+02]
 [-2.00100163e-04  7.65141782e-05  1.00000000e+00]]
23
[[ 1.19423878e+00  1.91713000e-02  1.37077129e+02]
 [-6.97017532e-02  1.26436795e+00  1.07220468e+02]
 [-1.53692071e-04  6.48434760e-05  1.00000000e+00]]
22
[[ 1.14370035e+00  6.27611018e-02  1.35601919e+02]
 [-9.84253362e-02  1.26985715e+00  1.08220112e+02]
 [-2.60480508e-04  9.81135395e-05  1.00000000e+00]]
25
[[ 1.16199250e+00  5.92429306e-02  1.35541589e+02]
 [-9

32
[[ 1.21275962e+00  2.21990894e-02  1.49488834e+02]
 [-8.04443921e-02  1.29890416e+00  1.10055404e+02]
 [-2.03511001e-04  3.63421787e-05  1.00000000e+00]]
23
[[ 1.25991444e+00  2.01401299e-02  1.49060131e+02]
 [-5.77706456e-02  1.31086515e+00  1.09021498e+02]
 [-1.13569425e-04  8.87512594e-05  1.00000000e+00]]
34
[[ 1.20355070e+00  4.94580175e-02  1.47576924e+02]
 [-8.78426275e-02  1.32189857e+00  1.06711983e+02]
 [-2.31269720e-04  8.22829864e-05  1.00000000e+00]]
27
[[ 1.24768106e+00  1.63864723e-01  1.37370171e+02]
 [-8.67586092e-02  1.38567657e+00  1.05305066e+02]
 [-2.16877166e-04  3.04584186e-04  1.00000000e+00]]
30
[[ 1.24197701e+00  4.96785956e-02  1.46412297e+02]
 [-7.31940798e-02  1.32040999e+00  1.06261084e+02]
 [-1.59650230e-04  1.05095682e-04  1.00000000e+00]]
29
[[ 1.21439983e+00  1.35831667e-02  1.48412570e+02]
 [-8.26758165e-02  1.26756738e+00  1.07916018e+02]
 [-1.94295019e-04 -7.92175222e-06  1.00000000e+00]]
25
[[ 1.21708167e+00  1.99128056e-02  1.48041947e+02]
 [-7